# INF8775 – Analyse et conception d’algorithmes
# TP3 – Automne 2024

Côté, Gabriel, 2082508

Tourigny, François, 2079718

Note finale:

<u>**Date limite de remise :**</u>  4 décembre 23h59 pour les deux groupes

# Instructions

## Rédaction et remise du rapport

- Ce notebook constitue à la fois le sujet du TP, votre code et votre rapport. Il contient déjà du code pour faciliter vos mesures et l'affichage de vos résultats, ainsi qu'un squelette pour votre rapport.

- Complétez directement le notebook, vous êtes libres de créer des nouvelles cellules de code ou de texte.

- Vous pouvez utiliser des fichiers externes pour stocker des exemplaires et des résultats, mais nous devons être capable de comprendre facilement votre démarche et de la reproduire.

- <u>**IMPORTANT**</u> Remettez le fichier du notebook sur Moodle avec le nom `MATRICULE1_MATRICULE2.ipynb`

- Vous pouvez inclure du code trouvé sur Internet, mais vous devez en mentionner la source, sous peine d'être sanctionnés pour plagiat.

## Mise en situation

Ce travail pratique se répartit sur deux séances de laboratoire et est une occasion de mettre en application les connaissances vues en cours. Vous devrez développer l'algorithme de votre choix pour essayer de résoudre le plus efficacement possible le problème donné. Une partie de la note sera accordée en fonction des résultats que vous obtiendrez par rapport aux autres équipes.

## Description du problème

Le problème qu'on vous demande de résoudre cette fois-ci est un peu plus difficile. Vous êtes responsable de la séparation des voteurs d'un pays en circonscriptions. Un des deux candidats principaux, M. T, vient vous voir et vous demande de lui garantir une victoire (pour un montant non négligeable d'argent). Évidemment, vous refusez, mais le problème est intéressant et vous décidez d'essayez de le résoudre. Si ça vous intéresse, ce problème est ce qu'on appelle du *gerrymandering*.

Vous aurez comme entrée à votre problème une carte du pays (représentée par une matrice carrée $n \times n$) qui contient à chaque position le nombre de voteurs pour candidat X (un chiffre entre 0 et 1000). Votre objectif est de créer $n$ circonscriptions de sorte à ce que candidat X gagne l'élection. Quelques spécifications importantes:

- La variable $n$ représente un côté de la matrice. Il y a donc, $n^2$ villes.
- Chaque position de la matrice représente une ville
- Chaque circonscription doit contenir **$n$ villes**. Une solution reste valide si une circonscription ne contient pas exactement $n$ villes, mais il y a une **pénalité** qui y est associée.
- Les villes d'une circonscription doivent être proche les unes des autres. On aimerait garder cette **distance à, au plus, $n/2$**. Encore une fois, on permet de briser cette contrainte, mais il y aura une pénalité qui y est associée. (**Distance manhattan**)
- Les villes d'une circonscription ne doivent **pas être nécessairement voisines** tant qu'on respecte la distance maximale. Voir l'exemple plus bas.
- Le candidat remporte une circonscription si le nombre de voteurs dans cette circonscription est supérieur à $500n$.
- Le candidat cherche à remporter le plus de circonscriptions qu'il peut.

![alt text](distance_example.png)


## Jeu de données

La classe Problem existe pour simplifier l'interface des différentes fonctions utilitaires. Elle permet de générer des jeux de données avec la méthode `generate_sample` ci-dessous. Elle génère une matrice carrée de taille $n$ contenant des nombres entre $1$ et $1000$. Vous pouvez utilisez des exemplaires aléatoires pour tester votre code. La compétition sera faite sur les mêmes exemplaires de tailles différentes pour toutes les équipes d'un même groupe.

In [ ]:
import random
from collections.abc import Iterable

def generate_city() -> int:
    return round(min(1000,max(0,random.normalvariate(450,200))))

class Problem():
    def __init__(self, size: int, num_samples: int = 5) -> None:
        self.size = size
        self.num_samples = num_samples

    def generate_sample(self) -> list[list[int]]:
        """Returns a matrix containing values between 0 and 1000. Each value is the number of voters in a given city"""
        return [[generate_city() for _ in range(self.size)] for _ in range(self.size)]

    def generate_dataset(self) -> Iterable[list[list[int]]]:
        """Returns an iterator over as many samples as are described"""
        return (self.generate_sample() for _ in range(self.num_samples))

# Implantations et expérimentations

Ces fonctions auxiliaires vous sont fournies pour vérifier l'exactitude des vos algorithmes, mesurer leurs performance et afficher vos résultats.

Il est recommandé de prendre le temps de lire et comprendre le code.

Exécutez la cellule ci-dessous pour pouvoir utiliser les fonctions auxiliaires.

In [ ]:
import matplotlib.pyplot as plt
import time
from collections.abc import Callable
from math import ceil
import math
from scipy.stats import linregress
import numpy as np
from tqdm import trange

class InvalidSolution(Exception):
    def __init__(self):
        super().__init__("Invalid solution, verify your code.")

class Measure():
    """A wrapper to contain information on taken measures"""
    def __init__(self, size: int, mean: int, score:int) -> None:
        self.size = size
        self.mean_score = score
        self.mean = mean

def score_solution(original: list[list[int]], solution: list[list[tuple[int,int]]]) -> int:
    """Returns the score of the current solution. The score function is a penalty that must be minimized."""
    return votes_score(original, solution) + size_score(solution) + distance_score(solution)

def votes_score(original: list[list[int]], solution: list[list[tuple[int,int]]]) -> int:
    """Calculates the part of the score associated to lost districts. 
    It is 5 times the square of the number of lost districts."""
    lost_districts = 0
    for district in solution:
        sum = 0
        for city in district:
            sum += original[city[0]][city[1]]
        if sum <= 500*len(district):
            lost_districts += 1
    return 5 * lost_districts**2

def size_score(solution: list[list[tuple[int,int]]]) -> int:
    """Calculates the part of the score associated to districts having the wrong size.
    It is the square of the difference between the wanted number of cities and the 
    current number of cities in a given district."""
    n = len(solution)
    size_penality = 0
    for district in solution:
        size_penality += (len(district)-n)**2
    return size_penality

def distance_score(solution: list[list[tuple[int,int]]]) -> int:
    """Calculates the part of the score associated to the distance between cities in a district.
    It is the mean square distance between each city and every other city in its district."""
    distance_score = 0
    n = len(solution)
    for district in solution:
        for i,city in enumerate(district):
            for j in range(i+1, len(district)):
                distance_score += (max(0, distance_manhattan(city, district[j])-ceil(n/2)))**2
    return distance_score/len(solution)

def distance_manhattan(city_a: tuple[int,int], city_b: tuple[int,int]) -> int:
    return abs(city_a[0] - city_b[0]) + abs(city_a[1] - city_b[1])

def is_valid_solution(original: list[list[int]], solution: list[list[tuple[int,int]]]) -> bool:
    """Validates solution"""
    n = len(original)

    if len(solution) != n:
        print(f"The solution does not contain {n} districts.")
        return False

    for district in solution:
        if len(district) < 1:
            print("The solution contains empty districts.")
            return False
        for city in district:
            if len(city)!=2:
                print("Solution must contain 2 coordinates per city.")
                return False
        for coord in city:
            if coord < 0 or coord >=n:
                print(f"City coordinates must below {n} and positive.")
                return False

    coord_set = set()
    for district in solution:
        for city in district:
            coord_set.add(city)
    if len(coord_set) != n*n:
        print(f"Solution contained {len(coord_set)} different cities while there should be {n*n} cities in the solution.")
        return False

    # Solution is valid
    return True

def make_problems(sizes: list[int], num_samples: int = 5) -> list[Problem]:
    """Creates problem instances using given sizes and max_numbers"""
    return [Problem(size,num_samples) for size in sizes]

def measure(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], sample: list[list[int]], time_scale: int = 1000) -> tuple[int,int]:
    """Returns a tuple containing the time as well as the score of the solution, in that order.
    
    Parameters:
        time_scale: Controls the level of precision of the time measurements.

    Raises:
        InvalidSolution: If the procedure returns an invalid solution, raises an exception.
    """
    start: int = time.time() * time_scale
    solution: list[int] = procedure(sample)
    end: int = time.time() * time_scale
    if not is_valid_solution(sample, solution):
        raise InvalidSolution()
    return (round(end - start), score_solution(sample, solution))

def measure_mean(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], prob: Problem, time_scale: int = 1000) -> Measure:
    """Generates multiple samples with the specified parameters and returns a Measure 
    instance representing the result as well as the problem.

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.
    """
    results = [measure(procedure,sample,time_scale) for sample in prob.generate_dataset()]
    mean_time = sum(result[0] for result in results) / prob.num_samples
    mean_score = sum(result[1] for result in results) / prob.num_samples
    return Measure(prob.size, mean_time, mean_score)

def measure_range(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], problems: list[Problem], time_scale: int = 1000) -> list[Measure]:
    """Measures the mean time taken for each problem in the given list.

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.

    Returns:
        A list of Measure instances containing the specifications
        of the problem as well as the mean time and the score.
    """
    return [
        measure_mean(procedure, prob, time_scale)
        for prob in problems
    ]

def display_data_as_table(measures: list[Measure]):
    """Prints a table with the data in the given list of measures"""
    print("{: <12} {: <12} {: <12}".format("Taille", "Temps moyen", "Score moyen"))
    for measure in measures:
        print("{: <12} {: <12} {: <12}".format(measure.size, measure.mean, measure.mean_score))

### The different tests are below, the names are in french to avoid confusion

def test_de_puissance(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de puissance"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    # Log both sets of values
    x = list(data.keys())
    y = list(data.values())

    # Perform the lin regression
    m, b, rvalue, _, _ = linregress(x, y)

    # Estimate the values of y based on the lin regression results
    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2f}x + {b:.2f}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

def test_de_rapport(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de rapport"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    plt.plot(x, y, label='Mesures')
    plt.scatter(x, y, label='Mesures')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()

def test_de_constantes(
    data: dict[int,int],
    x_label: str,
    y_label: str = "Temps (ms)",
    title: str = "Test de constantes"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    # Perform linear regression
    m, b, rvalue, _, _ = linregress(x, y)

    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2E}x + {b:.2E}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

# Algorithme

Votre algorithme sera en partie noté en fonction d'une évaluation relative entre les équipes. 4 points seront donnés aux équipes qui se classeront dans le premier quartile lors de notre évaluation sur un ensemble d'exemplaires. Les équipes se trouvant dans le quartile dont les algorithmes ont le moins bien performé recevront 1 point.

**IMPORTANT** Votre algo doit retourner une solution après 3 minutes. Si ce n'est pas le cas, vous serez pénalisé.

In [ ]:
class County():
    def __init__(self, row: int, col: int, pos: int, voters: int) -> None:
        self.row = row
        self.col = col
        self.pos = pos
        self.voters = voters
        self.neighbors = []
        self.group = -1

    def add_neighbor(self, neighbor):
        self.neighbors.append(neighbor)

    def get_neighbours(self):
        return self.neighbors
    
    def set_group(self, group: int):
        self.group = group
    
    def get_group(self):
        return self.group
    
    def is_in_group(self):
        return self.group != -1


In [ ]:
def get_neighbours_idx(idx, n):
    neighbours = []
    if idx - n > 0:
        neighbours.append(idx-n)
    if idx + n < n**2:
        neighbours.append(idx+n)
    if idx // n == (idx-1) // n:
        neighbours.append(idx-1)
    if idx // n == (idx+1) // n:
        neighbours.append(idx+1)
    return neighbours

def set_counties(sample) -> list[County]:
    n = len(sample)
    counties = [County(i, j, i*n+j, county) for i, line in enumerate(sample) for j, county in enumerate(line)]
    for county in counties:
        for neighbor_idx in get_neighbours_idx(county.pos, n):
            county.add_neighbor(counties[neighbor_idx])
    return counties


In [ ]:
def distance(county_row, county_col, center_row, center_col, n):
    return (abs(center_row - county_row) + abs(center_col - county_col))
    
def max_to_target(counties: list[County], target: int, n: int):
    solution = []
    counties_available = counties
    groups = []
    groups_center = []
    center_row = 0
    center_col = 0
    for group_id in trange(n):
        new_group = []
        number_items = 0
        center_row = 0
        center_col = 0
        group_neighbors = []
        while len(new_group) < n:
            selection = None
            if len(group_neighbors) == 0:
                selection = max(counties_available, key=lambda county: county.voters/n - distance(county.row, county.col, center_row, center_col, n)**2/n)
            elif sum(county.voters for county in new_group) < target:
                selection = max(group_neighbors, key=lambda county: county.voters/n - distance(county.row, county.col, center_row, center_col, n)**2/n)
                #selection = random.choice(group_neighbors)
            else:
                selection = min(group_neighbors, key=lambda county: county.voters/n + distance(county.row, county.col, center_row, center_col, n)**2/n)

            new_group.append(selection)
            center_row = (center_row * number_items + selection.row) / (number_items + 1)
            center_col = (center_col * number_items + selection.col) / (number_items + 1)
            number_items += 1
            selection.set_group(group_id)
            
            group_neighbors.extend([neighbor for neighbor in selection.neighbors if not neighbor.is_in_group() and not neighbor in group_neighbors])
            counties_available.remove(selection)
            if selection in group_neighbors:
                group_neighbors.remove(selection)
            
        groups.append(new_group)
        groups_center.append((center_row, center_col))
        solution.append([(county.row, county.col) for county in new_group])
    return solution, groups



In [ ]:

def main(sample):
    start_time = time.time()
    max_time = 60 * 3 - 1
    n = len(sample)
    target = 500 * n
    counties = set_counties(sample)
    print('initial solution')
    sol, groups = max_to_target(counties, target, n)
    if (time.time() - start_time) > max_time:
        return sol
    print("--- %s seconds for solution baseline ---" % (time.time() - start_time))

    #print('vote score: ', votes_score(sample, sol))
    #print('size score: ', size_score(sol))
    #print('distance score: ', distance_score(sol))
    #print('intial score: ',score_solution(sample, sol))
    #drawmap_of_discrits(n, sol)
    first_iter = True
    iter_time = time.time()
    time_to_iter = 0
    print('improvement')
    while(True):
        updated = amelioration_locale(groups, target, n, sample, sol)
        sol = updated

        if (time.time() - start_time) > max_time - time_to_iter:
            return sol
        if(first_iter):
            first_iter = False
            time_to_iter = (time.time() - iter_time)

    #print(" ---------------- AFTER UPDATE -----------------")
    #print('vote score: ', votes_score(sample, sol))
    #print('size score: ', size_score(sol))
    #print('distance score: ', distance_score(sol))
    #print('total score: ', score_solution(sample, sol))
    #drawmap_of_discrits(n, sol)
    return sol


def amelioration_locale(divisions, target, n, sample, sol):
    #init_score = score_solution(sample, sol)
    div_votes = [sum(county.voters for county in div) for div in divisions]
    groups_center = [(sum(county.row for county in div)/n, sum(county.col for county in div)/n) for div in divisions]

    #selected_div_idx = div_votes.index(max(div for div in div_votes if div < target))
    #selected_div_idx = divisions.index(random.choice([div for div in divisions if sum(county.voters for county in div) < target]))
    selected_div_idx = divisions.index(random.choice(divisions))
    #selected_div_idx = div_votes.index(min(div_votes))

    selected_div = divisions[selected_div_idx]
    center_swap_group_row = groups_center[selected_div_idx][0]
    center_swap_group_col = groups_center[selected_div_idx][1]

    modified_selected_div = [county.voters + distance(center_swap_group_row, center_swap_group_col, county.row, county.col, n) for county in selected_div]
    county_to_swap_idx = modified_selected_div.index(min(modified_selected_div))
    county_to_swap = divisions[selected_div_idx][county_to_swap_idx]
    #print('county to swap: ', county_to_swap.pos)
    differential_needed = target - county_to_swap.voters
    distance_div_swap = distance(center_swap_group_row, center_swap_group_col, county_to_swap.row, county_to_swap.col, n)

    swap_scores = {}
    for div_idx in range(len(divisions)):
        if div_idx == selected_div_idx:
            continue

        votes_to_spare = div_votes[div_idx] - target
        if votes_to_spare <= 0:
            votes_to_spare = target
        
        div_center_row = groups_center[div_idx][0]
        div_center_col = groups_center[div_idx][1]
        distance_div_swap_new_group = distance(div_center_row, div_center_col, county_to_swap.row, county_to_swap.col, n)

        for county_idx, county in enumerate(divisions[div_idx]):
            if county.voters - county_to_swap.voters > votes_to_spare:
                continue
            dist_county_curr_group = distance(div_center_row, div_center_col, county.row, county.col, n)
            dist_county_swap_group = distance(center_swap_group_row, center_swap_group_col, county.row, county.col, n)
            score_voters =  (county.voters/n - county_to_swap.voters/n) + (0 if county.voters - county_to_swap.voters < differential_needed else target)
            score_distance = (dist_county_curr_group + distance_div_swap) - (dist_county_swap_group + distance_div_swap_new_group)
            swap_scores[county.pos] = score_voters + n * score_distance

    #print(swap_scores)
    if len(swap_scores) == 0:
        return sol
    switch_county_pos = max(swap_scores, key=swap_scores.get)
    if swap_scores[switch_county_pos] < 0:
        return sol
    #print(switch_county_pos)
    switch_county_row = switch_county_pos // n
    switch_county_col = switch_county_pos % n

    div_switch_idx = -1
    pos_pair_switch = (switch_county_row, switch_county_col)
    for div_idx in range(len(divisions)):
        if pos_pair_switch in sol[div_idx]:
            div_switch_idx = div_idx
    
    #update solution
    sol[div_switch_idx].remove(pos_pair_switch)
    sol[div_switch_idx].append((county_to_swap.row, county_to_swap.col))

    sol[selected_div_idx].append(pos_pair_switch)
    sol[selected_div_idx].remove((county_to_swap.row, county_to_swap.col))

    #update divisions
    divisions[selected_div_idx].remove(county_to_swap)
    pos_arr = [div.pos for div in divisions[div_switch_idx]]

    idx_select = pos_arr.index(switch_county_pos)
    switch = divisions[div_switch_idx][idx_select]
    divisions[selected_div_idx].append(switch)

    divisions[div_switch_idx].append(county_to_swap)
    divisions[div_switch_idx].remove(switch)

    #print('new score: ',score_solution(sample, sol))
    #drawmap_of_discrits(n, sol)
    return sol
    




In [ ]:
resultats = measure_range(main, make_problems([100], 1) )

display_data_as_table(resultats)

In [ ]:
def drawmap_of_discrits(n, discrits: list[list[tuple[int,int]]]):
    colors = [[0 for _ in range(n)] for _ in range(n)]
    for i,district in enumerate(discrits):
        for city in district:
            colors[city[0]][city[1]] = i+1

    plt.imshow(colors, cmap='tab20')
    plt.show()


# Analyse asymptotique

Notre algorithme se sépare en 3 grande sections. La première d'entre elle est la création des objets County avoir leur voisinage. La seconde section est la génération d'une séparation initiale et la dernière partie est une amélioration locale.

# Création des County
Commençons par la création des County. La création se sépare en 2 phases, la première est de créer chaque County qui se fait en temps constant pour les n² éléments donc ceci nous donne theta(n²). La seconde étape de création est de passer par chacun des County créés et placer leur voisins qui sont au nombre maximal de 4 donc nous avons une complexité de theta(4*n²) qui est dans le même ordre de grandeur que theta(n²). Bref, la création des objets County se fait en theta(n² + n²) ce qui veut dire une complexité de theta(n²).

# Séparation initiale
La deuxième portion de notre algorithme est un algorithme glouton qui fait une répartion des County en n groupes. Dans chacun des n groupes, on fait n fois la recherche d'un prochain élément. Dans cette recherche de prochain élément, nous avons 2 choix qui sont de faire une rechreche parmis les voisins s'il y en a et sinon parmis tout les County qui ne sont pas déjà choisis dans un groupe. Dans le pire des cas, nous avons n² choix pour notre recherche de la valeur maximale. Bien que nous savons que la grande majorité des fois cette recherche est plutôt dans un ordre de grandeur n, le pire cas est n² donc nous mettons n² pour l'analyse théorique. Ce qui nous donne une complexité de theta(n\*n\*n²) ceci équivaut à une complexité de theta(n⁴).

# Amélioration locale
La dernière partie est une amélioration locale. L'amélioration se fait en plusieurs sections successives. La première de ces sections de l'amélioration locale est de calculer le centre du groupe nous avons n appels de la distance pour chacun des n groupes ce qui fait au total n² appels de la distance qui est un calcul en temps constant. Cette première partie de l'amélioration locale est donc en theta(n²\*1) donc theta(n²).
La seconde partie de l'amélioration locale est la sélection d'un County à échanger qui est en theta(n) pour le choix du groupe dans lequel prendre le County à échanger puis de theta(n) pour choisir le County parmis le groupe choisi. Ceci nous theta(n+n) donc theta(n) pour la sélection du County à échanger. Nous devons ensuite sélectionner le County avec lequel le premier chois sera échangé. Pour tous les County qui ne sont pas dans le groupe initial, nous générons un score de préférence pour faire l'échange avec le County choisi initialement. Ce calcul de score est en temps constant pour les au maximum n² County possibles pour faire un échange. Bref, le choix du second County pour l'échange se fait en theta(n²\*1) ce qui donne theta(n²).
Finalement, pour l'amélioration locale, nous avons l'échange des deux éléments qui se fait en temps constant theta(1).
Notre section d'amélioration locale se fait donc avec la complexité suivante theta(n²+n²+1) ce qui nous donne theta(n²)
La section d'amélioration locale est exécutée tant que du temps est disponible (avec une marge de sureté), mais ultimement le nombre d'exécutions peut être comme indépendant de n.

# Algorithme complet
Pour notre algorithme complet, nous pouvons additionner la complexité théorique de nos sections, car elle sont exécutées l'une à la suite de l'autre.
Nous avons donc theta(n²+n⁴+n²) ce qui nous donne une complexitée finale de theta(n⁴)

# Analyse hybride


PAS faire le test de constante (pour deux variable et on a pas)
Faire le test de puissance (faire en log log comme échelle) On devrait voir la valeur 4x
On peut utiliser un test de rapport pour confirmer le test de puissance
Faire le test de rapport en dernier si tout fonctionne


Expliquer l'utilisation de chacun des graphiques, expliquer en quoi le graphique justifie la complexité et pourquoi ne pas utiliser certain graphiques.

Effectuer une analyse hybride de votre algorithme.

In [ ]:
# On s'assure de générer toujours les mêmes problèmes pour pouvoir comparer les parties d'algorithme
problems = make_problems([x for x in range(11, 61)], 10)

### Algorithme greedy

#### Isolation de la solution greedy

Ici, on réécris notre algorithme sans l'amélioration locale, pour évaluer la complexité de notre solution initiale. Si on se souviens bien, notre hypothèse était une complexité d'au plus theta n^4 

In [ ]:
# On réécris certaines fonctions pour pouvoir les évaluer individuellement sans contrainte de temps maximum

def main_greedy(sample):
    start_time = time.time()
    n = len(sample)
    target = 500 * n
    counties = set_counties(sample)
    print('initial solution')
    sol, groups = max_to_target(counties, target, n)
    print("--- %s seconds for solution baseline ---" % (time.time() - start_time))
    return sol
    

#### Génération de moyennes

Ici on génère les moyennes de temps et de score pour chaque grandeur de n. On calcule à partir de n = 11 pour éviter le plus possible les problèmes de temps moyen inconsistant quand notre n est très petit. Pour plus de stabilité, nous faisons 10 runs par grandeur de n, pour au total 50 valeurs de n.

In [ ]:
resultats_greedy = measure_range(main_greedy, problems)

display_data_as_table(resultats_greedy)


#### Test de puissance

Ici, on utilise un test de puissance car on penses que notre algorithme greedy nous donne une complexité de theta(n^4), alors on essaie de confirmer que c'est bien le cas en regardant la pente de notre tendance sur le graphique log-log

In [ ]:
def dict_from_measures(measures: list[Measure]) -> dict:
    dict = {}
    for measure in measures:
        dict[np.log(measure.size)] = np.log(measure.mean)
    return dict

dict_greedy_sol = dict_from_measures(resultats_greedy)

test_de_puissance(dict_greedy_sol, "log de la grandeur de n", 'temps log moyen pour trouver la solution (ms)', "Test de puissance pour algorithme glouton")

Comme on peut voir sur le test de puissance, notre échelle log-log pour nos données nous donne une complexité autour de theta n^2.8, ce qui était attendu, mais beaucoup mieux que notre hypothèse de départ. Nous pensions que c'était theta(n^4), mais en pratique notre algorithme greedy performe beaucoup mieux que nous le pensions. 

### Test de rapport

Ici, on fait un test de rapport pour vérifier tout de même si notre hypothèse de départ peut être sensée, même si on sait maintenant que notre vraie complexité pratique est plutôt theta(n^2.8). C'est le dernier test que l'on considère pour cette partie, car on a pas besoin de faire le test de constance étant donné que nous n'avons qu'une variable dans notre algorithme

In [ ]:
import math
def change_to_predict(measures: list[Measure], hypothesis = lambda x: x) -> dict:
    new_dict = {}
    for key in measures:
        new_dict[key.size] = key.mean/hypothesis(key.size)
    return new_dict

test_de_rapport(change_to_predict(resultats_greedy, lambda x: x**4), "grandeur de n", "temps moyen pour trouver la solution divisé par n^4 (ms)" , "Test de rapport sur l'algorithme glouton")

Comme on peut le voir, on converge mais on se rapproche extrèmement de 0, ce qui indique qu'on surestime la valeur de notre complexité. C'est tout à fait normal vu qu'on vient de voir une valeur de theta(n^2.8) en pratique. Par contre, il est tout de même possible avec la différence de vitesse sur nos machines que la vraie complexité varie dépendamment de facteurs qu'on ne peut pas controller.

### Greedy et Amélioration locale avec un seul passage

Ici, on réécris notre algorithme avec amélioration locale, pour évaluer la complexité de notre solution finale. Si on se souviens bien, notre hypothèse était une complexité d'au plus theta n^4 

In [ ]:

def main_amelioration(sample):
    start_time = time.time()
    max_time = 60 * 3 - 1
    n = len(sample)
    target = 500 * n
    counties = set_counties(sample)
    print('initial solution')
    sol, groups = max_to_target(counties, target, n)
    if (time.time() - start_time) > max_time:
        return sol
    print("--- %s seconds for solution baseline ---" % (time.time() - start_time))

    first_iter = True
    iter_time = time.time()
    time_to_iter = 0
    updated = amelioration_locale(groups, target, n, sample, sol)
    sol = updated
    return sol

#### Génération de moyennes

Ici on génère les moyennes de la même façon

In [ ]:
resultats_am_locale = measure_range(main_amelioration, problems)

display_data_as_table(resultats_am_locale)

In [ ]:
dict_am_locale = dict_from_measures(resultats_am_locale)

test_de_puissance(dict_am_locale, "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de puissance Merge sort nombre d'éléments")

In [ ]:
test_de_rapport(change_to_predict(resultats_am_locale, lambda x: x**4), "rapport entre n log(n) et les augmentations de temps", "nombre d'items à balancer dans les deux listes combinées" , "Test de rapport sur l'algorithme glouton")

### Amélioration locale

Ceci était un essaie qui ne fonctionne pas de calculer indépendamment l'amélioration locale

In [ ]:
# def main_preparation(sample):
#     start_time = time.time()
#     n = len(sample)
#     target = 500 * n
#     counties = set_counties(sample)
#     sol, groups = max_to_target(counties, target, n)
#     return (sol, groups, sample, n, target, counties)
    

In [ ]:
# problems = make_problems([x for x in range(11, 15)], 10)
# resultats_initials = {"solutions_initiales": [],
#                        "groupes": [],
#                        "samples": [],
#                        "ns": [],
#                        "targets": [], 
#                        "counties": []}
# for prob in problems:
#     solutions_initiales, groupes, samples, ns, targets, counties = [], [], [], [], [], []
#     sols = [main_preparation(sample) for sample in prob.generate_dataset()]
#     for sol in sols:
#         solutions_initiales.append(sol[0])
#         groupes.append(sol[1])
#         samples.append(sol[2])
#         ns.append(sol[3])
#         targets.append(sol[4])
#         counties.append(sol[5])
    
#     resultats_initials["solutions_initiales"].append(solutions_initiales)
#     resultats_initials["groupes"].append(groupes)
#     resultats_initials["samples"].append(samples)
#     resultats_initials["ns"].append(ns)
#     resultats_initials["targets"].append(targets)
#     resultats_initials["counties"].append(counties)



In [ ]:

# def main_amelioration(sample, resultats_initials):
#     start_time = time.time()
#     max_time = 60 * 3 - 1
#     n = len(sample)
#     target = resultats_initials
#     counties = resultats_initials["counties"]
#     print('initial solution')
#     sol, groups = max_to_target(counties, target, n)
#     if (time.time() - start_time) > max_time:
#         return sol
#     print("--- %s seconds for solution baseline ---" % (time.time() - start_time))

#     first_iter = True
#     iter_time = time.time()
#     time_to_iter = 0
#     print('improvement')
#     while(True):
#         updated = amelioration_locale(groups, target, n, sample, sol)
#         sol = updated

In [ ]:
# print(resultats_initials["groupes"])

# resultats_am_locale = measure(lambda x: amelioration_locale(x["groupes"][0][0], x["targets"][0][0], x["ns"][0][0], x["samples"][0][0], x["solutions_initiales"][0][0]), resultats_initials)

# display_data_as_table(resultats_graphs)

In [ ]:
# def measure(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], sample: list[list[int]], time_scale: int = 1000) -> tuple[int,int]:
#     """Returns a tuple containing the time as well as the score of the solution, in that order.
    
#     Parameters:
#         time_scale: Controls the level of precision of the time measurements.

#     Raises:
#         InvalidSolution: If the procedure returns an invalid solution, raises an exception.
#     """
#     start: int = time.time() * time_scale
#     solution: list[int] = procedure(sample)
#     end: int = time.time() * time_scale
#     if not is_valid_solution(sample, solution):
#         raise InvalidSolution()
#     return (round(end - start), score_solution(sample, solution))

# def measure_mean(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], prob: Problem, greedy_results, time_scale: int = 1000) -> Measure:
#     """Generates multiple samples with the specified parameters and returns a Measure 
#     instance representing the result as well as the problem.

#     Raises:
#         InvalidSolution: If one of the samples results in an invalid solution.
#     """
#     results = [measure(procedure,sample,time_scale) for sample in prob.generate_dataset()]
#     mean_time = sum(result[0] for result in results) / prob.num_samples
#     mean_score = sum(result[1] for result in results) / prob.num_samples
#     return Measure(prob.size, mean_time, mean_score)

# def measure_range(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], problems: list[Problem], time_scale: int = 1000) -> list[Measure]:
#     """Measures the mean time taken for each problem in the given list.

#     Raises:
#         InvalidSolution: If one of the samples results in an invalid solution.

#     Returns:
#         A list of Measure instances containing the specifications
#         of the problem as well as the mean time and the score.
#     """
#     return [
#         measure_mean(procedure, problems[i], resultats_initials[], time_scale)
#         for i in len(problems)
#     ]

Nous avons essayé de faire l'analyse de la fonction d'amélioration locale indépendemment de l'algorithme principal. Par contre, comme l'amélioration locale demande une solution initiale sur laquelle faire l'amélioration et les fonctions fournies ne nous permettent pas isoler cette section de l'analyse. Par contre, nous avons fais l'analyse de la section de génération initale et aussi l'analyse de la génération initiale avec un passage d'amélioration locale. Nous voyons entre les deux analyse qu'il n'y a aucune différence d'ordre de grandeur. Ceci peux seulement nous borner supérieurement la complexité de notre amélioration locale qui est au pire dans le même ordre que la génération initiale. Nous savons que c'est O(n⁴) héoriquement et en pratique plus près de O(n³). Bref, un passage d'amélioration locale n'affecte pas l'ordre de grandeur, car il est au maximum aussi grand que nous génération de solution initiale.

# Analyse code carbon (2 pts)

Effectuer une anlayse code carbon en sélectionnant différent pays pour l'analyse. Commenter vos résultats.

In [ ]:
from codecarbon import EmissionsTracker, OfflineEmissionsTracker
try:
    tracker_USA = OfflineEmissionsTracker(measure_power_secs=5, tracking_mode="process", country_iso_code='USA')
    # tracker_CAD = OfflineEmissionsTracker(measure_power_secs=5, country_iso_code='CAN')
    # tracker_JPN = OfflineEmissionsTracker(measure_power_secs=5, country_iso_code='JPN')
    # tracker_FRA = OfflineEmissionsTracker(measure_power_secs=5, country_iso_code='FRA')
    # tracker_CHN = OfflineEmissionsTracker(measure_power_secs=5, country_iso_code='CHN')

    tracker_USA.start_task("main_USA")
    # tracker_CAD.start_task("main_CAD")
    # tracker_JPN.start_task("main_JPN")
    # tracker_FRA.start_task("main_FRA")
    # tracker_CHN.start_task("main_CHN")

    results = measure_range(main, make_problems([100], 1) )
    tracker_USA.stop_task()
    # tracker_CAD.stop_task()
    # tracker_JPN.stop_task()
    # tracker_FRA.stop_task()
    # tracker_CHN.stop_task()

finally:
    _ = tracker_USA.stop()
    # _ = tracker_CAD.stop()
    # _ = tracker_JPN.stop()
    # _ = tracker_FRA.stop()
    # _ = tracker_CHN.stop()

# Conclusion (6 pts)

# Étapes d'amélioration pour arriver à notre version actuelle
Nous avons commencé par un algorithme glouton qui n'avait aucune gestion du temps et aucune amélioration après le passage initial. Cet algorithme était en theta(n³) au lieu de theta(n⁴) comme notre algorithme actuel, car nous faisions une selection  du premier élément d'un groupe de manière complètment aléatoire. De plus, lorsqu'il n'y avait pas de voisins disponibles pour choisir le prochain élément, l'élément choisi dans le groupe était aléatoire ce qui donnait de mauvais scores de distance. De plus, nous n'avions pas encore l'objet County pour nous aider à clarifier le code. Du fait, nos heuristiques de choix de prochain élément étaient très rudimentaires et inefficace. Les score pour n=100 étaient autour de 700 000.

Nous avons par la suite ajouté une section d'amélioration locale. Pour choisir le groupe auquel nous voulions faire une amélioration, nous avons commencé par prendre celui qui a un nombre de vote le plus près d'une majorité et prendre le meilleur des échanges. Par contre, nous avons réalisé que certains des échanges étaient considérés comme sous optimaux et pris quand même. De plus, seulement un groupe recevait des modifications constantes jusqu'à atteindre un nombre de vote majoritaire puis un prochain. Nous avons donc changé notre algorithme pour prendre un groupe aléatoire parmis tous et vérifier que le meilleur échange améliore la situation et sinon ne pas faire d'échange. Ceci nous a permis de ne pas rester pris dans un minimum et presque toujours faire des améliorations avec chaque passage et arrêter seuelement lorsque le score obtenu est excellent.

# Points forts de notre algorithme
Notre algorithme trouve une solution initiale qui est souvent vraiment bonne et satisfaisante, même sans la mise en place d'une amélioration locale. Bref, bien que la complexité soit grande, notre passage initial donne une solution d'environ 25 000 pour n=100.

L'autre force de notre algorithme est que l'amélioration locale améliore rellement le score même après une grande quantité d'améliorations. En effet, le groupe qui reçoit une amélioration est choisi aléatoirement ce qui nous permet de tout améliorer. De plus, si jamais la quantité d'amélioration maximale pour la solution actuelle est atteinte, notre algorithme va préférer ne prendre aucune action plutôt qu'une action sous optimale.

# Points faibles de notre algorithme
Notre algorithme est vraiment lent pour trouver une solution initiale. Bien que la solution soit bonne, elle demande un trop grand temps de calcul pour de grandes valeurs de n. Un autre point faible de notre algorithme est de ne pas prendre avantage du débalancement possible dans les grandeurs des groupes de County. Nous pourrions surement aller chercher des scores plus élevés avec un débalancement faible dans le nombre de County inclus par groupe.

# Amélioration possibles
Pour améliorer notre algorithme, 2 avenues sont à explorer. La première est de fournir une version encore plus rapide de solution initiale pour être correct dans des situations ou le n prend une très très grande valeur. Ainsi je crois qu'il faudrait un système de thread pour faire des interruptions pour s'assurer de ne pas dépasser les temps prévus peu import la technique utilisée. Bien sur, l'utilisation de thread serait aussi très utile pour accélérer les performances et non seulement faire un meilleur timer.

La deuxième amélioration que nous aurions aimé explorer avec plus de temps est d'introduire un débalancement dans le nombre de County par groupe de vote. Ainsi, nous aurions pu réduire le score de groupes gagnés qui devient important avec des grandes valeurs par sa nature quadratique. Pour faire ce débalancement, l'algorithme d'amélioration locale pourrait avoir la capacité de prendre un County au lieu de simplement faire des échanges.

 ## Autres critères (2 pts)
 Qualité du code / 1 pt

Présentation générale / 1 pt

- Concision
- Qualité du français

Pénalité retard
- -2 pt / journée de retard, arrondi vers le haut. Les TPs ne sont plus acceptés après 3 jours.